In [1]:
import os
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
import nltk
from nltk.corpus import stopwords

# Only log error messages
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

import warnings
warnings.filterwarnings('ignore')

2023-05-09 20:44:37.971996: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Define Certain Variables

In [2]:
# The percentage of the dataset you want to split as train and test
TRAIN_TEST_SPLIT = 0.1

# Maximum length of the input to the model
MAX_INPUT_LENGTH = 1024  

# Minimum length of the output by the model
MIN_TARGET_LENGTH = 5  
# Maximum length of the output by the model
MAX_TARGET_LENGTH = 128  

# Batch-size for training our model
BATCH_SIZE = 8 

# Learning-rate for training our model 
LEARNING_RATE = 2e-5  

# Maximum number of epochs we will train the model for
MAX_EPOCHS = 10  

# This notebook is built on the t5-small checkpoint from the Hugging Face Model Hub
MODEL_CHECKPOINT = "t5-small"

## Load the dataset

The dataset consists of BBC articles and accompanying single sentence summaries. Specifically, each article is prefaced with an introductory sentence (aka summary) which is professionally written, typically by the author of the article. That dataset has 226,711 articles divided into training (90%, 204,045), validation (5%, 11,332), and test (5%, 11,334) sets. The dataset is available for download from [here](https://www.kaggle.com/pariza/bbc-news-summary).

The dataset has the following fields:

* **document**: the original BBC article to me summarized
* **summary**: the single sentence summary of the BBC article
* **id**: ID of the document-summary pair

In [3]:
from datasets import load_dataset, load_metric

# Load the dataset
raw_datasets = load_dataset("xsum", split="train")
print(raw_datasets)

Found cached dataset xsum (/Users/oabas/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})


In [4]:
raw_datasets[0]

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [5]:
raw_datasets.shape

(204045, 3)

In [6]:
# Split the dataset 
raw_datasets = raw_datasets.train_test_split(
    train_size=TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)

In [7]:
# # Split the dataset into train and test sets
# train_size = int(len(raw_datasets) * TRAIN_TEST_SPLIT)
# test_size = len(raw_datasets) - train_size
# train_dataset, test_dataset = raw_datasets.train_test_split(
#     test_size=test_size
# )

`len(raw_materials)` computes the length of the raw_materials dataset. `TRAIN_TEST_SPLIT` is a constant value that has been defined earlier, which represents the percentage of data that has to be used for training the model. 

`train_size` calculates the number of elements to be used for training by multiplying the length of the raw_materials dataset with the `TRAIN_TEST_SPLIT` value and typecasting it to an integer.

`test_size` calculates the number of data points that will be used for testing by subtracting the `train_size` from the original length of the `raw_materials` dataset.

Then the `train_test_split()` function is called on the `raw_materials` dataset using the `test_size` and `random_state` parameters. The `test_size` parameter specifies the size of the test dataset, while the `random_state` parameter shuffles the data randomly when split. The function returns two datasets, the train dataset and the test dataset, which are stored in the variables `train_dataset` and `test_dataset` respectively. 

Finally, the code splits the dataset into two parts: a training set and a test set, which can be used to train a machine learning model and evaluate its accuracy respectively.

## Data Pre-processing

In [8]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

I initializes a `tokenizer` object with a pre-trained model specified in the `MODEL_CHECKPOINT` variable.

Next, the code checks if the `MODEL_CHECKPOINT` variable is set to any of the T5 models: `"t5-small"`, `"t5-base"`, `"t5-large"`, `"t5-3b"`, `"t5-11b"`. If it is, the `prefix` variable is set to `"summarize: "`. Otherwise, `prefix` remains an empty string.

In [9]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

This code defines a function called `preprocess_function` that takes a dictionary of examples as input. The function first concatenates a prefix string to each document in the examples. It then tokenizes the concatenated documents into a format suitable for input to a model, using a pre-defined tokenizer with a maximum input length of `MAX_INPUT_LENGTH`. The function also tokenizes the summaries in the examples using the same tokenizer and a maximum target length of `MAX_TARGET_LENGTH,` but in "target" mode. The function then adds the summary labels to the tokenized model inputs and returns the resulting dictionary of model inputs.

In [10]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
# tokenized_datasets = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/20404 [00:00<?, ? examples/s]

/Users/oabas/opt/anaconda3/envs/data_science/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/20405 [00:00<?, ? examples/s]

This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

## Defining the model

In [11]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

2023-05-09 20:47:01.371262: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


For training Sequence to Sequence models, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels. Thus, we use the `DataCollatorForSeq2Seq` provided by the Hugging Face Transformers library on our dataset. The `return_tensors='tf'` ensures that we get `tf.Tensor` objects back.

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [13]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns = ["input_ids", "attention_mask", "labels"],
    shuffle = True,
    batch_size = BATCH_SIZE,
    collate_fn = data_collator,
)

test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns = ["input_ids", "attention_mask", "labels"],
    shuffle = False,
    batch_size = BATCH_SIZE,
    collate_fn = data_collator,
)

generation_dataset = (
    tokenized_datasets["test"].
    shuffle(seed=42).
    select(list(range(200))).
    to_tf_dataset(
        columns = ["input_ids", "attention_mask", "labels"],
        shuffle = False,
        batch_size = BATCH_SIZE,
        collate_fn = data_collator,
    )
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


The code creates three TensorFlow datasets for a natural language processing task. The first dataset is the `training dataset` which shuffles the data and batches it using the specified batch size. The second dataset is the `test dataset` which does not shuffle the data and also batches it using the specified batch size. The final dataset is a `generation dataset` which selects a subset of the `test data`, shuffles that subset and uses it to generate text. All datasets contain columns for input IDs, attention masks and labels, and use the specified `data_collator` function to collate the data into a batch.

## Building and Compiling the the model

In [17]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss=None)

## Training and Evaluating the model

To evaluate our model on-the-fly while training, we will define `metric_fn` which will calculate the `ROUGE` score between the groud-truth and predictions.

In [20]:
import keras_nlp

rouge_l = keras_nlp.metrics.RougeL()

def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    # We will print only the F1 score, you can use other aggregation metrics as well
    result = {"RougeL": result["f1_score"]}

    return result

In [22]:
# Now we can finally start training our model!
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]

# For now we will use our test set as our validation_data
model.fit(
    train_dataset, validation_data=test_dataset, 
    epochs=MAX_EPOCHS, callbacks=callbacks
)

Epoch 1/10


TypeError: in user code:

    File "/Users/oabas/opt/anaconda3/envs/data_science/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/oabas/opt/anaconda3/envs/data_science/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/oabas/opt/anaconda3/envs/data_science/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/oabas/opt/anaconda3/envs/data_science/lib/python3.10/site-packages/transformers/modeling_tf_utils.py", line 1604, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/Users/oabas/opt/anaconda3/envs/data_science/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/optimizer_v2.py", line 576, in minimize
        grads_and_